<a href="https://colab.research.google.com/github/HamzaGbada/Numba-cuda/blob/main/Numba_CUDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

فَلَو أَنَّ ما أَسعى لِأَدنى مَعيشَةٍ **** كَفاني وَلَم أَطلُب قَليلٌ مِنَ المالِ 

وَلَكِنَّما أَسعى لِمَجدٍ مُؤَثَّلٍ **** وَقَد يُدرِكُ المَجدَ المُؤَثَّلَ أَمثالي

امرؤ القيس

In this notebook, we will discuss how to do parallel computing on your GPU with CUDA using python!

The [NVIDIA® CUDA® Toolkit](https://developer.nvidia.com/cuda-toolkit) provides a development environment for creating high performance GPU-accelerated applications.

We will cover these sections:

1.   Cuda kernel
2.   Memory management
3.   Device function
4.   Threads and blocks configuration
5.   Application in image processing



Before we start, let's take a look on what is [Numba](https://numba.pydata.org/).

So Numba is a just-in-time compiler that translates python code into machine code using [LLVM](https://llvm.org/), which makes numerical alogrithms execution speed in Python comparable to C or FORTRAN algorithms.
 


# Cuda Kernel

As a first we must check [CUDA programming terminology](https://numba.pydata.org/numba-doc/dev/cuda/overview.html#terminology), let's take a minimal example where we add 2 for each element of a vector

In [1]:
from numba import cuda

@cuda.jit
def add_gpu(x, out):
  idx = cuda.grid(1)
  out[idx] = x[idx] + 2

Let's discuss this code in some details:

When writing CUDA kernel, we must declare the input and the output of the kernel as argment.
But, What's `cuda.grid(1)`? 

`cuda.grid(1)` returns the unique index for the current thread in the whole grid. With N threads, idx will range from 0 to N. We will discuss thread indexing in more details in the section 4.



Now, we setup our data

In [2]:
import numpy as np

a = np.arange(10,dtype=np.float32)

# send input vector to the device
d_a = cuda.to_device(a)

# create output vector on the device
d_out = cuda.device_array_like(d_a)

In [3]:
# we decide to use 2 blocks, each containing 5 threads for our vector 
nbr_block_per_grid = 2
nbr_thread_per_block = 5
add_gpu[nbr_block_per_grid, nbr_thread_per_block](d_a, d_out)

In [4]:
# now we get our output
out = d_out.copy_to_host()
out

array([ 2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11.], dtype=float32)

Le Voilà!

Let's compare it to a CPU function

In [5]:
def add_cpu(x):
   for i in range(x.size):
     x[i]+=2
   return x

In [6]:
%timeit add_gpu[nbr_block_per_grid, nbr_thread_per_block](d_a, d_out)

The slowest run took 8.00 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 5: 136 µs per loop


In [7]:
%timeit add_cpu(a)

10000 loops, best of 5: 24.9 µs per loop


 Wait! What is that! The CPU version is about 6 time faster!
 
Let's understand what is happening together. When running on the GPU, the following steps are performed :

1. the input data (the array a ) is transferred to the GPU memory;
the calculation of the square root is done in parallel on the GPU for all elements of a
2. the resulting array is sent back to the host system.

If the calculation is too simple, the time spent in calculation is negligible compared to the data transfer time. 


Let's see what happens with a more involved calculation.

In [8]:
import numpy as np

a = np.arange(32768,dtype=np.float32)

# send input vector to the device
d_a = cuda.to_device(a)

# create output vector on the device
d_out = cuda.device_array_like(d_a)

# we decide to use 2 blocks, each containing 5 threads for our vector 
nbr_block_per_grid = 32
nbr_thread_per_block = 1024
add_gpu[nbr_block_per_grid, nbr_thread_per_block](d_a, d_out)

%timeit add_gpu[nbr_block_per_grid, nbr_thread_per_block](d_a, d_out)

10000 loops, best of 5: 145 µs per loop


In [9]:
%timeit add_cpu(a)

10 loops, best of 5: 81.1 ms per loop


Now, it's clear! when we increase the size of our vector, the calculation became more complex and so the GPU version became faster!

**Conclusion:** When dealing with simple data and calculation, it's a waste of time to use GPU.
 

# Memory management

In the last section, we have seen a little example of memory management without any explication. So the main idea of this section is to learn how to deal with data transfers between ***Host*** and ***Device***.

It is advisable to minimize data transfers to improve performance and this can be done with device arrays. 

To show this in simple way, we will use an example of normalizing pixel range of an image from [0,255] into [0,10], our calculation will be based on this formula: 

$j =  10 \times \frac{i}{255} \;\;\; \forall 0 \leq i\leq 255 $

where i is the intensity of the initiale image and j is the output.

After that, we will perfom a Gamma Correction based on this function:

$f(V)=V^\alpha \: \: \: \; \alpha>0$


Before we start let's see how they operate in CPU

In [1]:
import numpy as np

n = 512

# The image will be generated randomly as numpy array
image = np.random.randint(256, size=(n, n))

def normlize_image(image):
  return image*10 / 255

def gamma_correction(image, gamma):
  image = np.power(image / float(10), gamma)
  return image * 10

In [2]:
%%timeit
gamma = 0.5

norm_img = normlize_image(image)

gamma_corr = gamma_correction(image, gamma)

100 loops, best of 5: 17.3 ms per loop


Now, we will implement the cuda kernel of the functions above,

In [1]:
from numba import cuda
from operator import pow 
import numpy as np

@cuda.jit
def normalize_image_gpu(image, out):

  i, j = cuda.grid(2)
  out[i,j] = ((image[i,j])/(255)) * 10
  
@cuda.jit
def gamma_correction_gpu(image, out, gamma):
  i, j = cuda.grid(2)
  out[i,j] = pow(image[i,j], gamma) 
  

So what we should do:

1. transfer the image to device
2. Create an intermediate arrays on device (an empty array with the same features as image array)
3.  Create the output array on device ( for gamma correction output) with same shape as the input but with different data type (numpy.float because gamma coefficient is float)

In [4]:
gamma = 0.5

blockdim = (32, 32)
griddim = (image.shape[0] // blockdim[0] + 1, image.shape[1] // blockdim[1] + 1)
# data transfer to device
d_image = cuda.to_device(image)
d_normlized = cuda.device_array_like(image)
d_gamma = cuda.device_array((image.shape[0], image.shape[1]), dtype=np.float)

In [5]:
%%timeit
normalize_image_gpu[griddim, blockdim](d_image, d_normlized)
gamma_correction_gpu[griddim, blockdim](d_normlized, d_gamma, gamma)

The slowest run took 918.07 times longer than the fastest. This could mean that an intermediate result is being cached.
1 loop, best of 5: 282 µs per loop


Finally, here is how to retrieve the gamma correction result:

In [6]:
gamma_result = d_gamma.copy_to_host()
gamma_result

array([[2.        , 2.        , 1.41421356, ..., 3.        , 2.82842712,
        0.        ],
       [1.73205081, 2.        , 2.82842712, ..., 2.44948974, 2.        ,
        0.        ],
       [2.44948974, 2.44948974, 1.73205081, ..., 1.73205081, 3.16227766,
        2.23606798],
       ...,
       [1.41421356, 3.        , 1.73205081, ..., 2.82842712, 1.41421356,
        2.82842712],
       [1.        , 2.        , 2.44948974, ..., 1.        , 1.41421356,
        2.        ],
       [1.41421356, 2.64575131, 0.        , ..., 2.        , 2.64575131,
        3.        ]])

# Device function

In the examples above we have seen how to write cuda kernel of simple functions, but what if we need to write more complex function and we faced a problem where we need to call other function, what we should do? we have two solution:

1. Implement everything in the same kernel and this solution seem to be bad, it made our code a Spaghetti code.
2. Create multiple cuda kernel and call them inside other kernel, this [feature](http://developer.download.nvidia.com/GTC/PDF/GTC2012/PresentationPDF/S0338-GTC2012-CUDA-Programming-Model.pdf) is call [*dynamic parallelism*](https://developer.nvidia.com/blog/cuda-dynamic-parallelism-api-principles/), but unfortunately Numba does not support it yet. 

So, we need another solution, here **Device function** appear!

CUDA device functions can only be invoked from within the device (by a kernel or another device function). 
Unlike a kernel function, a device function can return a value like normal functions.


Let's write a device function that's similar to *numpy.max*.

Note: *numpy.max* is not supported by Numba check [this](https://numba.pydata.org/numba-doc/dev/reference/numpysupported.html) to see supported NumPy features.


In [2]:
@cuda.jit(device=True)
def maximum_device(arr):
  max = arr[0,0]
  for i in range(arr.shape[0]):
    for j in range(arr.shape[1]):
      if arr[i,j]>=max:
        max = arr[i,j]
  return max

In [5]:
import numpy as np

n = 512

image = np.random.randint(256, size=(n, n))

maximum_device(image)

TypeError: ignored

Ah yes I forget! Device function should only be declared inside CUDA kernel. We will modify our Gamma Correction Kernel!

In [6]:
@cuda.jit
def gamma_correction_device(image, out, gamma):
  i, j = cuda.grid(2)
  maximum = maximum_device(image)
  out[i,j] = pow(image[i,j], gamma) / maximum

gamma = 0.5

blockdim = (32, 32)
griddim = (image.shape[0] // blockdim[0] + 1, image.shape[1] // blockdim[1] + 1)
# data transfer to device
d_image = cuda.to_device(image)
d_normlized = cuda.device_array_like(image)
d_gamma = cuda.device_array((image.shape[0], image.shape[1]), dtype=np.float)

In [7]:
%%timeit
normalize_image_gpu[griddim, blockdim](d_image, d_normlized)
gamma_correction_device[griddim, blockdim](d_normlized, d_gamma, gamma)

The slowest run took 1536.68 times longer than the fastest. This could mean that an intermediate result is being cached.
1 loop, best of 5: 289 µs per loop


As we can see device function does not affect on the execution time.

The device function allows us to avoid code duplication.

Obviously, it is a bit artifial to use a device function in such an easy case. But when implementing complex algorithms, these functions can prove very useful.